In [0]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, SeparableConv2D, MaxPooling3D, Conv3D, DepthwiseConv2D

In [2]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 240/Project/Data

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/CSE 240/Project/Data


Spectrograms/


In [0]:
img = load_img('Spectrograms/train/concentration/S001E02_AF3.png')

In [5]:
print(type(img))
print(img.format)
print(img.mode)
print(img.size)
print(img)

<class 'PIL.Image.Image'>
None
RGB
(432, 288)
<PIL.Image.Image image mode=RGB size=432x288 at 0x7EFF23E8DC18>


In [6]:
print(type(img))
# convert to numpy array
img_array = img_to_array(img)
print(img_array.dtype)
print(img_array.shape)
print(img_array[0][0])
# convert back to image
img_pil = array_to_img(img_array)
print(type(img))
#for i in range(288):
  #for j in range(432):
    #print(img_array[i][j])

<class 'PIL.Image.Image'>
float32
(288, 432, 3)
[255. 255. 255.]
<class 'PIL.Image.Image'>


In [0]:
import os
import numpy as np
import pandas as pd

In [8]:
base_path = "Spectrograms/concentration"
concentration_data_array = []
for foldername in os.listdir(base_path):
  print(foldername)
  array = np.zeros((288,432,1))
  for filename in os.listdir(os.path.join(base_path, foldername)):
    example = foldername
    channel = filename.split('.')[0].split("_")[1]
    filepath = os.path.join(os.path.join(base_path, foldername), filename)
    #print(filename,filepath)
    img = load_img(filepath)
    img_array = img_to_array(img)
    #print(img_array.shape, array.shape)
    array = np.concatenate((array, img_array), axis=2)
    #print(img_array.shape, array.shape)
  concentration_data_array.append(np.delete(array,0,2))
#print(data_array['S025E02'].shape)
concentration_data_array = np.array(concentration_data_array)
print(concentration_data_array.shape)

S005E04
S003E04
S002E04
S001E04
S001E02
S006E04
S007E04
S008E04
S009E04
S010E04
S011E04
S013E04
S012E04
S014E04
S015E04
S016E04
S017E04
S018E04
S019E04
S020E04
S021E04
S022E04
S023E04
S024E04
S025E04
S026E04
S027E04
S028E04
S029E04
S030E04
S002E02
S003E02
S004E02
S005E02
S006E02
S007E02
S008E02
S009E02
S010E02
S011E02
S012E02
S013E02
S014E02
S015E02
S016E02
S017E02
S018E02
S019E02
S020E02
S021E02
S022E02
S023E02
S024E02
S025E02
S026E02
S027E02
S028E02
S029E02
S030E02
S004E04
(60, 288, 432, 42)


In [9]:
base_path = "Spectrograms/relaxed"
relaxed_data_array = []
for foldername in os.listdir(base_path):
  print(foldername)
  array = np.zeros((288,432,1))
  for filename in os.listdir(os.path.join(base_path, foldername)):
    example = foldername
    channel = filename.split('.')[0].split("_")[1]
    filepath = os.path.join(os.path.join(base_path, foldername), filename)
    #print(filename,filepath)
    img = load_img(filepath)
    img_array = img_to_array(img)
    #print(img_array.shape, array.shape)
    array = np.concatenate((array, img_array), axis=2)
    #print(img_array.shape, array.shape)
  relaxed_data_array.append(np.delete(array,0,2))
#print(data_array['S025E02'].shape)
relaxed_data_array = np.array(relaxed_data_array)
print(relaxed_data_array.shape)

S001E01
S001E03
S002E01
S002E03
S003E01
S004E01
S005E01
S006E01
S007E01
S008E01
S009E01
S010E01
S011E01
S012E01
S013E01
S014E01
S015E01
S016E01
S017E01
S018E01
S019E01
S020E01
S021E01
S022E01
S023E01
S024E01
S025E01
S026E01
S027E01
S028E01
S029E01
S030E01
S003E03
S004E03
S005E03
S006E03
S007E03
S008E03
S009E03
S010E03
S011E03
S012E03
S013E03
S014E03
S015E03
S016E03
S017E03
S018E03
S019E03
S020E03
S021E03
S022E03
S023E03
S025E03
S024E03
S026E03
S027E03
S028E03
S029E03
S030E03
(60, 288, 432, 42)


In [0]:
X = np.concatenate((relaxed_data_array, concentration_data_array))
relaxed_data_array, concentration_data_array = None, None

In [11]:

Y = np.concatenate((np.zeros((60,1)),np.ones((60,1))))
print(X.shape, Y.shape)

(120, 288, 432, 42) (120, 1)


In [0]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X, Y = None, None

(102, 288, 432, 42) (18, 288, 432, 42) (102, 1) (18, 1)


In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)
print(X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape)

(86, 288, 432, 42) (18, 288, 432, 42) (16, 288, 432, 42) (86, 1) (18, 1) (16, 1)


In [15]:
print(X_train[4][200][0])

[255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.]


In [16]:
datagen = ImageDataGenerator()
train_base_path = "Spectrograms/train"
test_base_path = "Spectrograms/test"
validation_base_path = "Spectrograms/validation"
train_generator = datagen.flow_from_directory(
        train_base_path,
        target_size=(288, 432),
        class_mode='binary')
test_generator = datagen.flow_from_directory(
        test_base_path,
        target_size=(288, 432),
        class_mode='binary')
validation_generator = datagen.flow_from_directory(
        validation_base_path,
        target_size=(288, 432),
        class_mode='binary')

Found 1344 images belonging to 2 classes.
Found 168 images belonging to 2 classes.
Found 168 images belonging to 2 classes.


In [10]:
# confirm the iterator works
batchX, batchy = train_generator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
batchX, batchy = validation_generator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batch shape=(32, 288, 432, 3), min=0.000, max=255.000
Batch shape=(32, 288, 432, 3), min=0.000, max=255.000


In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D

#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(288, 432, 42)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [0]:
adam = optimizers.Adam(lr=0.00000001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
model.fit_generator(train_generator, epochs=10, steps_per_epoch=42, validation_data=validation_generator, validation_steps=6)

Epoch 1/10
42/42 [==============================] - 195s 5s/step - loss: 7.8170 - acc: 0.5097 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 2/10
42/42 [==============================] - 250s 6s/step - loss: 8.2796 - acc: 0.4807 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 3/10
42/42 [==============================] - 206s 5s/step - loss: 7.8051 - acc: 0.5104 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 4/10
42/42 [==============================] - 203s 5s/step - loss: 7.8763 - acc: 0.5060 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 5/10
42/42 [==============================] - 196s 5s/step - loss: 8.1610 - acc: 0.4881 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 6/10
42/42 [==============================] - 217s 5s/step - loss: 7.7814 - acc: 0.5119 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 7/10
42/42 [==============================] - 236s 6s/step - loss: 8.1373 - acc: 0.4896 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 8/10
42/42 [==============================] - 203s 5s/step - loss: 7.8170 - a

In [40]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=8, shuffle=True)

Train on 86 samples, validate on 16 samples
Epoch 1/50
86/86 [==============================] - 3s 30ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 2/50
86/86 [==============================] - 3s 30ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 3/50
86/86 [==============================] - 3s 30ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 4/50
86/86 [==============================] - 3s 30ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 5/50
86/86 [==============================] - 3s 30ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 6/50
86/86 [==============================] - 3s 30ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 7/50
86/86 [==============================] - 3s 31ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 8/50
86/86 [=====================

In [0]:
# evaluate model
loss = model.evaluate_generator(test_generator, steps=6)

In [33]:
from keras import optimizers
#create model
model2 = Sequential()
#add model layers
model2.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(288, 432, 42)))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(32, kernel_size=3, activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten())
model2.add(Dense(50, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.00000001, momentum=0.5)
model2.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=8, shuffle=True)

Train on 86 samples, validate on 16 samples
Epoch 1/50
86/86 [==============================] - 3s 38ms/step - loss: 7.1770 - acc: 0.5000 - val_loss: 6.2674 - val_acc: 0.3750
Epoch 2/50
86/86 [==============================] - 2s 29ms/step - loss: 8.2435 - acc: 0.4302 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 3/50
86/86 [==============================] - 3s 29ms/step - loss: 6.5705 - acc: 0.5349 - val_loss: 5.3310 - val_acc: 0.6250
Epoch 4/50
86/86 [==============================] - 2s 29ms/step - loss: 5.8166 - acc: 0.5116 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 5/50
86/86 [==============================] - 2s 29ms/step - loss: 6.7826 - acc: 0.5116 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 6/50
86/86 [==============================] - 2s 29ms/step - loss: 5.9418 - acc: 0.5349 - val_loss: 6.0385 - val_acc: 0.5625
Epoch 7/50
86/86 [==============================] - 2s 29ms/step - loss: 6.2927 - acc: 0.5581 - val_loss: 5.3176 - val_acc: 0.6250
Epoch 8/50
86/86 [=====================

In [44]:

model3 = Sequential()
#add model layers
model3.add(SeparableConv2D(256, kernel_size=3, activation='relu', input_shape=(288, 432, 42)))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Conv2D(64, kernel_size=3, activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Flatten())
model3.add(Dense(50, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.00000001, momentum=0.5)
model3.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model3.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=8, shuffle=True)

Train on 86 samples, validate on 16 samples
Epoch 1/50
86/86 [==============================] - 6s 72ms/step - loss: 6.8861 - acc: 0.5000 - val_loss: 5.4613 - val_acc: 0.3750
Epoch 2/50
86/86 [==============================] - 5s 54ms/step - loss: 2.1409 - acc: 0.5465 - val_loss: 0.7774 - val_acc: 0.6875
Epoch 3/50
86/86 [==============================] - 5s 54ms/step - loss: 1.2870 - acc: 0.5116 - val_loss: 0.7876 - val_acc: 0.6875
Epoch 4/50
86/86 [==============================] - 5s 54ms/step - loss: 1.3109 - acc: 0.5581 - val_loss: 0.7722 - val_acc: 0.6875
Epoch 5/50
86/86 [==============================] - 5s 54ms/step - loss: 1.2268 - acc: 0.5465 - val_loss: 0.7586 - val_acc: 0.6875
Epoch 6/50
86/86 [==============================] - 5s 54ms/step - loss: 1.2203 - acc: 0.5465 - val_loss: 0.7533 - val_acc: 0.6875
Epoch 7/50
86/86 [==============================] - 5s 53ms/step - loss: 1.2423 - acc: 0.5116 - val_loss: 0.7895 - val_acc: 0.7500
Epoch 8/50
86/86 [=====================

In [46]:
model4 = Sequential()
#add model layers
model4.add(SeparableConv2D(256, kernel_size=5, activation='relu', input_shape=(288, 432, 42)))
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(SeparableConv2D(64, kernel_size=3, activation='relu'))
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Flatten())
model4.add(Dense(50, activation='relu'))
model4.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.00000001, momentum=0.5)
model4.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model4.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=8, shuffle=True)

Train on 86 samples, validate on 16 samples
Epoch 1/50
86/86 [==============================] - 6s 75ms/step - loss: 1.1216 - acc: 0.5000 - val_loss: 1.0257 - val_acc: 0.6250
Epoch 2/50
86/86 [==============================] - 5s 60ms/step - loss: 1.0565 - acc: 0.5000 - val_loss: 0.9893 - val_acc: 0.6250
Epoch 3/50
86/86 [==============================] - 5s 60ms/step - loss: 0.9977 - acc: 0.5000 - val_loss: 0.9556 - val_acc: 0.6250
Epoch 4/50
86/86 [==============================] - 5s 60ms/step - loss: 0.9466 - acc: 0.5000 - val_loss: 0.9273 - val_acc: 0.6250
Epoch 5/50
86/86 [==============================] - 5s 60ms/step - loss: 0.9026 - acc: 0.5000 - val_loss: 0.9054 - val_acc: 0.6250
Epoch 6/50
86/86 [==============================] - 5s 60ms/step - loss: 0.8642 - acc: 0.5000 - val_loss: 0.8890 - val_acc: 0.6250
Epoch 7/50
86/86 [==============================] - 5s 60ms/step - loss: 0.8323 - acc: 0.4884 - val_loss: 0.8760 - val_acc: 0.6250
Epoch 8/50
86/86 [=====================

In [49]:
model5 = Sequential()
#add model layers
model5.add(Conv3D(256, kernel_size=3, activation='relu', input_shape=(288, 432, 42)))
model5.add(MaxPooling3D(pool_size=(2, 2, 2)))
model5.add(Conv2D(64, kernel_size=3, activation='relu'))
model5.add(MaxPooling2D(pool_size=(2, 2)))
model5.add(Flatten())
model5.add(Dense(50, activation='relu'))
model5.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.00000001, momentum=0.5)
model5.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model5.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=8, shuffle=True)

ValueError: ignored

In [50]:
from sklearn import svm
model5 = svm.SVC()
model5.fit(X_train, y_train)

ValueError: ignored

In [60]:
model6 = Sequential()
#add model layers
model6.add(DepthwiseConv2D(kernel_size=(3,3), activation='relu', input_shape=(288, 432, 42)))
model6.add(MaxPooling2D(pool_size=(2, 2)))
model6.add(DepthwiseConv2D(kernel_size=(3,3), activation='relu'))
model6.add(MaxPooling2D(pool_size=(2, 2)))
model6.add(Flatten())
model6.add(Dense(50, activation='relu'))
model6.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.00000001, momentum=0.5)
model6.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model6.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=150, batch_size=8, shuffle=True)

Train on 86 samples, validate on 16 samples
Epoch 1/150
86/86 [==============================] - 5s 62ms/step - loss: 1.7861 - acc: 0.5000 - val_loss: 2.4542 - val_acc: 0.3750
Epoch 2/150
86/86 [==============================] - 4s 49ms/step - loss: 1.7211 - acc: 0.5000 - val_loss: 2.3723 - val_acc: 0.3750
Epoch 3/150
86/86 [==============================] - 4s 49ms/step - loss: 1.6565 - acc: 0.5000 - val_loss: 2.2836 - val_acc: 0.3750
Epoch 4/150
86/86 [==============================] - 4s 49ms/step - loss: 1.5939 - acc: 0.5000 - val_loss: 2.2029 - val_acc: 0.3750
Epoch 5/150
86/86 [==============================] - 4s 49ms/step - loss: 1.5336 - acc: 0.5000 - val_loss: 2.1223 - val_acc: 0.3750
Epoch 6/150
86/86 [==============================] - 4s 49ms/step - loss: 1.4753 - acc: 0.5000 - val_loss: 2.0438 - val_acc: 0.3750
Epoch 7/150
86/86 [==============================] - 4s 49ms/step - loss: 1.4188 - acc: 0.5000 - val_loss: 1.9698 - val_acc: 0.3750
Epoch 8/150
86/86 [=============

In [34]:
metrics = model2.evaluate(X_test,y_test)
print(metrics)

18/18 [==============================] - 1s 33ms/step
[6.268148422241211, 0.6111111044883728]


In [41]:
metrics = model.evaluate(X_test,y_test)
print(metrics)

18/18 [==============================] - 0s 22ms/step
[9.742568969726562, 0.3888888955116272]


In [45]:
metrics = model3.evaluate(X_test,y_test)
print(metrics)

18/18 [==============================] - 1s 35ms/step
[0.9793123602867126, 0.6666666865348816]


In [47]:
metrics = model4.evaluate(X_test,y_test)
print(metrics)

18/18 [==============================] - 1s 31ms/step
[0.6619123220443726, 0.6111111044883728]


In [59]:
metrics = model6.evaluate(X_test,y_test)
print(metrics)

18/18 [==============================] - 0s 21ms/step
[0.8770436644554138, 0.3888888955116272]


In [61]:
# higher epochs version
metrics = model6.evaluate(X_test,y_test)
print(metrics)

18/18 [==============================] - 0s 22ms/step
[0.8298603892326355, 0.3888888955116272]
